In [1]:
# import 
import numpy as np
import pandas as pd
import os
from math import sqrt
from pathlib import Path
from tqdm import tqdm
tqdm.pandas()

import datetime

In [2]:
class DataSet:
    # クラス変数の定義
    DRIVE_DIR = r'/content/drive/MyDrive/Colab Notebooks/kaggle/H_and_M_Personalized_Fashion_Recommendations'
    INPUT_DIR = os.path.join(DRIVE_DIR, 'input')

    def __init__(self) -> None:
        # インスタンス変数(属性の初期化)
        self.ALL_ITEMS = []
        self.ALL_USERS = []
        self.df_val: pd.DataFrame
        pass

    def read_data(self, c_id_short: bool = True):

        # ファイルパスを用意
        csv_train = os.path.join(DataSet.INPUT_DIR, 'transactions_train.csv')
        csv_sub = os.path.join(DataSet.INPUT_DIR, 'sample_submission.csv')
        csv_users = os.path.join(DataSet.INPUT_DIR, 'customers.csv')
        csv_items = os.path.join(DataSet.INPUT_DIR, 'articles.csv')

        # データをDataFrame型で読み込み
        if c_id_short == True:
            # 実際の購買記録の情報
            self.df = pd.read_parquet(os.path.join(
                DataSet.DRIVE_DIR, 'transactions_train.parquet'))
            # dfのcustomer_idはshort版に加工されてるから、カラム名を変更しておく
            self.df.rename(
                columns={'customer_id': 'customer_id_short'}, inplace=True)

            # dfのarticle_idを文字列に為ておく?
            # 各顧客の情報(メタデータ)
            self.dfu = pd.read_parquet(os.path.join(
                DataSet.DRIVE_DIR, 'customers.parquet'))
            self.dfu.rename(
                columns={'customer_id': 'customer_id_short'}, inplace=True)
            # 各商品の情報(メタデータ)
            self.dfi = pd.read_parquet(os.path.join(
                DataSet.DRIVE_DIR, 'articles.parquet'))
        else:
            self.df = pd.read_csv(csv_train, dtype={'article_id': str},
                                  parse_dates=['t_dat']  # datetime型で読み込み
                                  )
            self.dfu = pd.read_csv(csv_users)  # 各顧客の情報(メタデータ)
            self.dfi = pd.read_csv(
                csv_items, dtype={'article_id': str})  # 各商品の情報(メタデータ)

            # customer_id_shortカラムを生成
            self.df['customer_id_short'] = self.df["customer_id"].apply(lambda s: int(s[-16:], 16)).astype("uint64")
            self.dfu['customer_id_short'] =self.dfu["customer_id"].apply(lambda s: int(s[-16:], 16)).astype("uint64")

        # price カラムを×10^3しておく...その方が、小数点以下と整数で分けやすい??
        self.df['price'] = self.df['price'] * (10 **3)

        # 提出用のサンプル
        self.df_sub = pd.read_csv(csv_sub)
        

        # customer_idカラムのみのpd.DataFrameを作っておく(たぶん色々便利なので)
        self.df_sub["customer_id_short"] = pd.DataFrame(
            self.df_sub["customer_id"].apply(lambda s: int(s[-16:], 16))).astype("uint64")
        self.cid = pd.DataFrame(self.df_sub["customer_id_short"])

    def read_data_sampled(self, sampling_percentage: float = 5):
        # ファイルパスを用意
        sampled_data_dir = os.path.join(DataSet.INPUT_DIR, 'sampling_dir')
        path_transactions = os.path.join(
            sampled_data_dir, f'transactions_train_sample{sampling_percentage}.csv.gz')
        path_article = os.path.join(
            sampled_data_dir, f'articles_train_sample{sampling_percentage}.csv.gz')
        path_customers = os.path.join(
            sampled_data_dir, f'customers_sample{sampling_percentage}.csv.gz')

        # インスタンス変数として読み込み
        self.df = pd.read_csv(path_transactions,
                              dtype={'article_id': str},
                              parse_dates=['t_dat']  # datetime型で読み込み
                              )
        # price カラムを×10^3しておく...その方が、小数点以下と整数で分けやすい??
        self.df['price'] = self.df['price'] * (10 **3)
        self.dfi = pd.read_csv(path_article, dtype={'article_id': str})
        self.dfu = pd.read_csv(path_customers)
        # df_subはそのまま
        csv_sub = os.path.join(DataSet.INPUT_DIR, 'sample_submission.csv')
        self.df_sub = pd.read_csv(csv_sub)
        # customer_id_shortカラムを作る.
        self.df_sub["customer_id_short"] = pd.DataFrame(
            self.df_sub["customer_id"].apply(lambda s: int(s[-16:], 16))).astype("uint64")

        # customer_idカラムのみのpd.DataFrameを作っておく(たぶん色々便利なので)
        self.cid = pd.DataFrame(self.dfu["customer_id_short"].copy())
        print(self.cid)

In [3]:
Colab_bool = False
# Load data
if Colab_bool==False:
    df_t = pd.read_csv(r'C:\Users\Masat\デスクトップ_Instead\webアプリ開発\H_and_M_Personalized_Fashion_Recommendations\input\transactions_train_sample5.csv.gz')
    df_i = pd.read_csv(r'C:\Users\Masat\デスクトップ_Instead\webアプリ開発\H_and_M_Personalized_Fashion_Recommendations\input\articles_train_sample5.csv.gz')
    df_u = pd.read_csv(r'C:\Users\Masat\デスクトップ_Instead\webアプリ開発\H_and_M_Personalized_Fashion_Recommendations\input\customers_sample5.csv.gz')

# 本番環境(=colab)では...
if Colab_bool :
    # DataSetオブジェクトの読み込み
    dataset = DataSet()
    # DataFrameとしてデータ読み込み
    dataset.read_data(c_id_short=True)

    # データをDataFrame型で読み込み
    df_t = dataset.df
    df_sub = dataset.df_sub # 提出用のサンプル
    df_u = dataset.dfu # 各顧客の情報(メタデータ)
    df_i = dataset.dfi # 各商品の情報(メタデータ)

# datetime型に変換
df_t['t_dat'] = pd.to_datetime(df_t['t_dat'])

In [4]:
# merge
df_t = pd.merge(
    df_t, df_i, on='article_id', how='left'
)

df_t = pd.merge(
    df_t, df_u, on='customer_id_short', how='left'
)


In [7]:
df_t.head()

,t_dat,customer_id_short,article_id,price,sales_channel_id,week,product_code,prod_name,product_type_no,product_type_name,...,section_name,garment_group_no,garment_group_name,detail_desc,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,2018-09-20,52397916724644664,638282001,0.022017,2,0,638282,33213,67,39,...,29,1019,1,41754,-1,-1,1,0,32,36155
1,2018-09-20,52397916724644664,684033003,0.030492,2,0,684033,12397,78,26,...,25,1019,1,20690,-1,-1,1,0,32,36155
2,2018-09-20,52397916724644664,617903013,0.022017,2,0,617903,1855,255,3,...,17,1005,0,2120,-1,-1,1,0,32,36155
3,2018-09-20,52397916724644664,617903009,0.022017,2,0,617903,1855,255,3,...,17,1005,0,2120,-1,-1,1,0,32,36155
4,2018-09-20,52397916724644664,661435002,0.084729,2,0,661435,7894,272,0,...,37,1016,11,3703,-1,-1,1,0,32,36155


In [6]:
# 週毎の売上集計
df_t.groupby(pd.Grouper(key='t_dat', freq="W"))['customer_id_short'].count()
# 月毎の売上集計
df_t.groupby(pd.Grouper(key='t_dat', freq="MS"))['customer_id_short'].count()


t_dat
2018-09-01    29754
2018-10-01    69634
2018-11-01    62438
2018-12-01    57735
2019-01-01    63457
2019-02-01    56607
2019-03-01    65308
2019-04-01    73313
2019-05-01    77657
2019-06-01    97503
2019-07-01    88606
2019-08-01    62694
2019-09-01    61315
2019-10-01    56847
2019-11-01    59173
2019-12-01    54447
2020-01-01    53814
2020-02-01    51148
2020-03-01    51426
2020-04-01    66166
2020-05-01    68881
2020-06-01    88436
2020-07-01    67601
2020-08-01    61898
2020-09-01    39092
Freq: MS, Name: customer_id_short, dtype: int64

# ターゲットエンコーディング特徴量

- Target Encoding（Target Mean Encoding）とはカテゴリカル（質的）データを数値に変換する方法の 1 つ。
- 様々な手法があるのですが、Target Encoding の一番の特徴は目的変数を使用するという点。
- 筆者の言葉で誤解を恐れずに言うのであれば Target Encoding が生み出すのは「値が大きいほど目的変数の値も大きい確率が高い」特徴量ということになる。
- 目的変数という答えを利用する Target Encoding はデータセットによっては非常に強力な力を持つ。

参考
- https://www.codexa.net/target_encoding/


## ターゲットエンコーディングの基本的な考え方

- 問題を単純にするため、このエントリでは二値分類問題に限定して考える。 
- 二値分類問題における Target Encoding では、一般的に**説明変数に含まれるカテゴリ変数ごとの、目的変数の平均値**を特徴量として用いる。 
  - カテゴリ変数は、複数の組み合わせになることもある。 
- また、平均値を用いる手法は、より限定的に **Target Mean Encoding **と呼称することもある。 
  - Target Encoding 自体は目的変数を用いた特徴量抽出の手法全般に対する呼称と理解してるけど、一般的には Target Mean Encoding を指すことが多い気がする。

## ターゲットエンコーディングの手法について

参考
- CatBoost: unbiased boosting with categorical features (PDF)

前述した CatBoost の論文には、Targe Encoding の手法として以下の 4 つが紹介されている。

手法の名前に共通で含まれる TS は Target Statistics の略となっている。

- Greedy TS
- Leave-one-out TS
- Holdout TS
- Ordered TS

上記の中で、Greedy TS と Leave-one-out TS はリークが生じるため使うべきではない。 
そのため、**一般的には Holdout TS が用いられている**。
 Ordered TS は CatBoost の論文の中で提案されている手法で、リークが生じにくいとされている。

## 下準備

サンプル用のデータフレームを用意する。 色々なフルーツと、それが美味しいかを示しているとでも考えてもらえれば。

In [10]:
data = {
'category': ['apple', 'apple',
'banana', 'banana', 'banana',
'cherry', 'cherry', 'cherry', 'cherry',
'durian'],
'label': [0, 1,
0, 0, 1,
0, 1, 1, 1,
1],
}

df = pd.DataFrame(data=data)
df

,category,label
0,apple,0
1,apple,1
2,banana,0
3,banana,0
4,banana,1
5,cherry,0
6,cherry,1
7,cherry,1
8,cherry,1
9,durian,1


上記を見ると、なんとなく cherry は美味しい割合が高そうで banana は低そうと感じるはず。 この、ラベルの割合が高そう低そう、というのが実は正に Target Encoding の考え方になる。

## Greedy TS(使っちゃダメ)

まず最初に示すのは Greedy TS から。 最初に断っておくと、この手法はリークを起こすため使ってはいけない。

Greedy TS では、データ全体で計算したカテゴリ変数ごとの目的変数の平均値がそのまま特徴量になる。 つまり、以下のようにカテゴリごとに集計した平均値となる。

In [11]:
ts = df.groupby('category', as_index=False).agg({'label': 'mean'})
ts

,category,label
0,apple,0.500000
1,banana,0.333333
2,cherry,0.750000
3,durian,1.000000


元のデータに特徴量を追加する場合、次のようになる。 基本的に、同じカテゴリは同じ特徴量になる。

In [14]:
pd.merge(df, ts, on='category', how='left')

,category,label_x,label_y
0,apple,0,0.500000
1,apple,1,0.500000
2,banana,0,0.333333
3,banana,0,0.333333
4,banana,1,0.333333
5,cherry,0,0.750000
6,cherry,1,0.750000
7,cherry,1,0.750000
8,cherry,1,0.750000
9,durian,1,1.000000


上記の Greedy TS は特徴量を付与するデータ自体も集計対象としている。 そのため、本来は使えない目的変数の情報が説明変数に漏れてしまっている。 結果として、Local CV で性能を高く見積もってしまうことになる。

## Leave-one-out TS(これも使っちゃダメ)

続いては Leave-one-out TS という手法。 一見すると上手くいきそうだけど、このやり方もリークが生じるため使ってはいけない。

まず、Leave-one-out TS の基本的な考え方は、**特徴量を付与する対象となるデータをピンポイントで除いて集計する**というもの。 

計算方法にはいくつかやり方があるけど、ここではあらかじめ集計した値から付与対象のデータを取り除く方法を取る。

まずはカテゴリ変数ごとの目的変数の合計とカウントを計算しておく。

In [16]:
agg_df = df.groupby('category').agg({'label': ['sum', 'count']})
agg_df

label      
           sum count
category            
apple        1     2
banana       1     3
cherry       3     4
durian       1     1

上記の集計から、付与する対象のデータだけを除外して計算した平均値を計算する関数を定義する。

In [23]:
def loo_ts(row):
    """apply関数用。

    Parameters
    ----------
    row : _type_
        _description_

    Returns
    -------
    _type_
        _description_
    """
    # 処理対象レコードのカテゴリに対する、「カテゴリ変数ごとの目的変数の合計値とカウント」の集計を取り出す
    group_ts = agg_df.loc[row.category]
    # 取り出した「目的変数の合計値」から、処理対象レコードの目的変数の値を除く
    loo_sum = group_ts.loc[('label', 'sum')] - row.label
    # 取り出した「目的変数のカウント」から、処理対象レコードの存在を除く
    loo_count = group_ts.loc[('label', 'count')] - 1
    # 合計値をカウントで割って平均を取り出す( = 処理対象レコードのみを除いてTarget mean encodingしている)
    return loo_sum / loo_count

上記の関数を各行に適用して得られる結果が次の通り。 

これが Leave-one-out TS の特徴量となる。 

先ほどの結果と違って同じカテゴリの中でも特徴量の値が異なっていることがわかる。

In [18]:
ts = df.apply(loo_ts, axis=1)
ts

C:\Users\Masat\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in longlong_scalars
  if __name__ == '__main__':


0    1.000000
1    0.000000
2    0.500000
3    0.500000
4    0.000000
5    1.000000
6    0.666667
7    0.666667
8    0.666667
9         NaN
dtype: float64

このやり方のまずさは元の説明変数と結合してみるとわかる。 
以下で、例えば apple のカテゴリの結果は目的変数を反転させた結果となっていることがわかる。 
もちろん、これは極端なパターンだけど、**これでは目的変数をそのまま説明変数に埋め込んでいる**のと変わりがない。

In [22]:
ts.name = 'loo_ts'
df.join(ts)

,category,label,loo_ts
0,apple,0,1.000000
1,apple,1,0.000000
2,banana,0,0.500000
3,banana,0,0.500000
4,banana,1,0.000000
5,cherry,0,1.000000
6,cherry,1,0.666667
7,cherry,1,0.666667
8,cherry,1,0.666667
9,durian,1,NaN


ちなみに durian の NaN は Leave-one-out しようにも、同じカテゴリのデータがないために生じている。 これを回避するには、**分母 (と場合によっては分子にも) に定数を加える Smoothing** をした方が良い。 以下では分かりやすさのために足して引いてしている。

## 

In [26]:
def loo_ts(row):
    """apply関数用。

    Parameters
    ----------
    row : _type_
        _description_

    Returns
    -------
    _type_
        _description_
    """
    # 処理対象レコードのカテゴリに対する、「カテゴリ変数ごとの目的変数の合計値とカウント」の集計を取り出す
    group_ts = agg_df.loc[row.category]
    # 取り出した「目的変数の合計値」から、処理対象レコードの目的変数の値を除く
    loo_sum = group_ts.loc[('label', 'sum')] - row.label
    # 取り出した「目的変数のカウント」から、処理対象レコードの存在を除く
    loo_count = group_ts.loc[('label', 'count')] - 1
    # 合計値をカウントで割って平均を取り出す( = 処理対象レコードのみを除いてTarget mean encodingしている)
    return loo_sum / (loo_count + 1) # smoothing

df.apply(func=loo_ts, axis=1)

0    0.500000
1    0.000000
2    0.333333
3    0.333333
4    0.000000
5    0.750000
6    0.500000
7    0.500000
8    0.500000
9    0.000000
dtype: float64

Smoothingによって、今度は NaN が登場しない。

## Holdout TS

- 続いて紹介するのが、現在一般的な Target Encoding として用いられている Holdout TS という手法。- 
- より厳密には Holdout TS を交差させて全データに適用したもの。 
- Holdout TS は、前述した 2 つの手法よりもリークが起こりにくいとされる (起こらないわけではない)。

Holdout TS では、**Leave-one-out TS ではひとつだけだった除外データを増やす**。 つまり、**特定の割合でデータを学習用とホールドアウトに分割**することになる。 その上で、学習用のデータを用いて計算した平均値をホールドアウトの特徴量として使う。 これを全データに対して k-Fold CV の要領で適用すれば良い。

計算方法は、Leave-one-out TS と同じように、あらかじめ集計した値から除外対象を引くやり方にしてみる。 まずは単純に合計とカウントを集計する。

In [27]:
agg_df = df.groupby('category').agg({'label': ['sum', 'count']})
agg_df

label      
           sum count
category            
apple        1     2
banana       1     3
cherry       3     4
durian       1     1

データを分割するための KFold オブジェクトを用意する。

In [30]:
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

folds

StratifiedKFold(n_splits=3, random_state=42, shuffle=True)

生成した特徴量を入れる Series オブジェクトを用意しておく。

In [29]:
import numpy as np
ts = pd.Series(data=np.empty(df.shape[0]), index=df.index)
ts

0     0.000000e+00
1     0.000000e+00
2    4.940656e-324
3    4.940656e-324
4    4.940656e-324
5    9.881313e-324
6    9.881313e-324
7    9.881313e-324
8    9.881313e-324
9    1.482197e-323
dtype: float64

そして、次のようにしてホールドアウト分を全体から除外した上で平均値を計算する

In [31]:
for _, holdout_idx in folds.split(X=df, y=df['label']):
    # ホールドアウトする行を取り出す
    holdout_df = df.iloc[holdout_idx]
    # ホールドアウトしたデータで合計とカウントを計算する
    holdout_agg_df = holdout_df.groupby(
        'category').agg({'label': ['sum', 'count']})
    # 全体の集計(目的変数の合計値＆カウント)からホールドアウトした分を引く
    train_agg_df = agg_df - holdout_agg_df
    # ホールドアウトしたデータの平均値を計算していく
    oof_ts = holdout_df.apply(lambda row: train_agg_df.loc[row.category][('label', 'sum')]
                              / train_agg_df.loc[row.category][('label', 'count')], axis=1)
    # 生成した特徴量を記録する
    ts[oof_ts.index] = oof_ts


C:\Users\Masat\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [32]:
ts.name = 'holdout_ts'
df.join(ts)

,category,label,holdout_ts
0,apple,0,1.000000
1,apple,1,0.000000
2,banana,0,0.500000
3,banana,0,0.000000
4,banana,1,0.000000
5,cherry,0,1.000000
6,cherry,1,0.500000
7,cherry,1,0.666667
8,cherry,1,0.500000
9,durian,1,NaN


しかし、上記では NaN となっている値が多いことに気づく。 これは、データの分割方法によっては学習データが少なくなって平均値が計算できなくなってしまうため。

Holdout TS でも、やはり Smoothing はした方が良さそう。

In [33]:
for _, holdout_idx in folds.split(X=df, y=df['label']):
    # ホールドアウトする行を取り出す
    holdout_df = df.iloc[holdout_idx]
    # ホールドアウトしたデータで合計とカウントを計算する
    holdout_agg_df = holdout_df.groupby(
        'category').agg({'label': ['sum', 'count']})
    # 全体の集計(目的変数の合計値＆カウント)からホールドアウトした分を引く
    train_agg_df = agg_df - holdout_agg_df
    # ホールドアウトしたデータの平均値を計算していく
    oof_ts = holdout_df.apply(lambda row: train_agg_df.loc[row.category][('label', 'sum')]
                              / (train_agg_df.loc[row.category][('label', 'count')]+1), axis=1)
    # 生成した特徴量を記録する
    ts[oof_ts.index] = oof_ts

In [34]:
ts.name = 'holdout_ts'
df.join(ts)

,category,label,holdout_ts
0,apple,0,0.500000
1,apple,1,0.000000
2,banana,0,0.333333
3,banana,0,0.000000
4,banana,1,0.000000
5,cherry,0,0.750000
6,cherry,1,0.333333
7,cherry,1,0.500000
8,cherry,1,0.333333
9,durian,1,0.000000


## Ordered TS

最後に紹介するのが CatBoost の論文で提案されている Ordered TS というやり方。 このやり方は Holdout TS よりも、さらにリークを起こしにくいらしい。

Ordered TS の基本的な考え方はオンライン学習に着想を得たもの。 

- ある行の特徴量として平均値を計算するのに、その時点で過去に登場したデータの集計を用いる。 
- ようするにストリーミング的にデータが次々と到着する場面で、到着したデータには過去の平均値を付与していくのをイメージすると良い。
- データが到着する毎に、過去のデータ (History) も増えて平均値も更新されていく

しかし、上記の考え方はデータに時系列の要素が含まれないことも多い点が問題となる。 そこで、Ordered TS では **artificial "time" (人工的な時間) という概念を持ち込む**。 これは、**ようするにデータが登場する順番を人工的に定義したもの**。 典型的には、データのインデックス番号をランダムにシャッフルして使えば良い。

説明が長くなってもあれなのでコードに移る。 まずはデータフレームのインデックスを元に artificial "time" を定義する。

In [35]:
np.random.seed(42)
artificial_time = np.random.permutation(df.index)
artificial_time

array([8, 1, 5, 0, 7, 2, 9, 4, 3, 6], dtype=int64)

続いて、グループ化するのに使うカラムとターゲットのカラム、Smoothing の有無について変数を用意しておく。

In [ ]:
group_col = 'category'
target_col = 'label'
smooth = False